<a href="https://colab.research.google.com/github/btwwang1/btwwang1/blob/main/bus_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#bus tracker citybus only
import requests
import json
import datetime
import pytz
import pandas as pd

route = input("Enter the bus route: ")
directions = ('inbound', 'outbound')
stopid_list_main = []
stop_name_list_main = []
check = True

for bound in directions:
    url = f"https://rt.data.gov.hk/v2/transport/citybus/route-stop/CTB/{route}/{bound}"
    response = requests.get(url)
    if response.status_code == 200:
        stopid_list = []
        # Parse the JSON response and extract the stop information
        data = json.loads(response.text)
        route_list = data["data"]
        # Display the stop information to the user
        for stop in route_list:
            stopid_list.append(stop['stop'])
        if stopid_list == []:
          print("There is no such route operated by CTB")
          check = False
          break
        stopid_list_main.append(stopid_list)
        stop_name_list = {}
        for stop_id in stopid_list:
            url_stop = f"https://rt.data.gov.hk/v2/transport/citybus/stop/{stop_id}"
            response_stop = requests.get(url_stop)
            if response_stop.status_code == 200:
                # Parse the JSON response and extract the stop information
                data_stop = json.loads(response_stop.text)
                stop_name_list[stop_id] = data_stop["data"]["name_en"]
                # Display the stop information to the user
        print(f"Stop names for route {route} departing from {stop_name_list[stopid_list[0]]} to {stop_name_list[stopid_list[-1]]}:")
            # Create a Pandas DataFrame from the data
        stop_table = {"Stops ID": list(stop_name_list.keys()), "Stop Names": list(stop_name_list.values())}
        df = pd.DataFrame(stop_table)
        # Set the 'Name' column as the index
        df.set_index('Stops ID', inplace=True)

        # Print the DataFrame as a table
        print(df)
        stop_name_list_main.append(stop_name_list)
    else:
        print(f"Error: Unable to retrieve routing for route {route}")

while check == True:
    trace_stop = input("Please input the stop id u would like to trace: ")
    if trace_stop in stopid_list_main[0] or stopid_list_main[1]:
        eta_url = f"https://rt.data.gov.hk/v2/transport/citybus/eta/CTB/{trace_stop}/{route}"
        eta_response = requests.get(eta_url)
        if eta_response.status_code == 200:
            retrieved = json.loads(eta_response.text)
            data = retrieved["data"]
            eta_list_o = []
            eta_list_i = []
            for entries in data:
                if entries["rmk_en"] != "KMB Cycle" and entries["dir"] == "O" and entries["route"] == route:
                    eta_form = datetime.datetime.fromisoformat(entries["eta"])
                    eta_list_o.append(eta_form)
                elif entries["rmk_en"] != "KMB Cycle" and entries["dir"] == "I" and entries["route"] == route:
                    eta_form = datetime.datetime.fromisoformat(entries["eta"])
                    eta_list_i.append(eta_form)
            remaining_time_o = []
            remaining_time_i = []
            for eta in eta_list_o:
                remaining_time_o.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
            for eta in eta_list_i:
                remaining_time_i.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
            remaining_time_main = (remaining_time_i, remaining_time_o)
            remaining_time_i = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_i]
            remaining_time_o = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_o]
            if remaining_time_i != []:
              for remaining_time in remaining_time_i:
                  print(f"Remaining time for route {route} to {list((stop_name_list_main[0]).values())[-1]} at stop {stop_name_list_main[0][trace_stop]} is {remaining_time}")
            elif remaining_time_o != []:
              for remaining_time in remaining_time_o:
                  print(f"Remaining time for route {route} to {list((stop_name_list_main[1]).values())[-1]} at stop {stop_name_list_main[1][trace_stop]} is {remaining_time}")
            else:
              print("There is no bus operating at this stop in the coming hour.")
        else:
            print(f'Error: Unable to retrieve ETAs for route {route}')

        check = input("Do you want to trace another Stop? T/F ")
        check = check.upper()
        if check == "T":
            check = True
            continue
        else:
            break
    else:
        trace_stop = input("Please enter again the stop: ")

In [ ]:
#bus tracker (citybus) v.2
import requests
import json
import datetime
import pytz
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

route = input("Enter the bus route: ")
directions = ('inbound', 'outbound')
stop_name_list_main = []
check = True

def handle_click(index, bound):
    # Get the associated row data
    stop_name = df_main[bound].iloc[index]['Stop Names']
    stop_id = df_main[bound].iloc[index]['Stop IDs']
    display_eta(stop_name, stop_id, route, bound)

def display_eta(stop_name, stop_id, route, bound):
    eta_url = f"https://rt.data.gov.hk/v2/transport/citybus/eta/CTB/{stop_id}/{route}"
    eta_response = requests.get(eta_url)
    if eta_response.status_code == 200:
        retrieved = json.loads(eta_response.text)
        data = retrieved["data"]
        eta_list_o = []
        eta_list_i = []
        for entries in data:
            if entries["rmk_en"] != "KMB Cycle" and entries["dir"] == "O" and entries["route"] == route:
                eta_form = datetime.datetime.fromisoformat(entries["eta"])
                eta_list_o.append(eta_form)
            elif entries["rmk_en"] != "KMB Cycle" and entries["dir"] == "I" and entries["route"] == route:
                eta_form = datetime.datetime.fromisoformat(entries["eta"])
                eta_list_i.append(eta_form)
        remaining_time_o = []
        remaining_time_i = []
        for eta in eta_list_o:
            remaining_time_o.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
        for eta in eta_list_i:
            remaining_time_i.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
        remaining_time_main = (remaining_time_i, remaining_time_o)
        remaining_time_i = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_i]
        remaining_time_o = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_o]
        if remaining_time_i != []:
            for remaining_time in remaining_time_i:
                print(f"Remaining time for route {route} to {list((stop_name_list_main[0]).values())[-1]} at stop {stop_name} is {remaining_time}")
        elif remaining_time_o != []:
            for remaining_time in remaining_time_o:
                print(f"Remaining time for route {route} to {list((stop_name_list_main[1]).values())[-1]} at stop {stop_name} is {remaining_time}")
        else:
            print("There is no bus operating at this stop in the coming hour.")
    else:
        print(f'Error: Unable to retrieve ETAs for route {route}')

stop_table_main = {}
df_main = {}
for bound in directions:
    url = f"https://rt.data.gov.hk/v2/transport/citybus/route-stop/CTB/{route}/{bound}"
    response = requests.get(url)
    if response.status_code == 200:
        stop_name_list = {}
        # Parse the JSON response and extract the stop information
        data = json.loads(response.text)
        route_list = data["data"]
        # Display the stop information to the user
        for stop in route_list:
            stop_id = stop['stop']
            url_stop = f"https://rt.data.gov.hk/v2/transport/citybus/stop/{stop_id}"
            response_stop = requests.get(url_stop)
            if response_stop.status_code == 200:
                # Parse the JSON response and extract the stop information
                data_stop = json.loads(response_stop.text)
                stop_name_list[stop_id] = data_stop["data"]["name_en"]
                # Display the stop information to the user
        stop_name_list_main.append(stop_name_list)
        if stop_name_list == {}:
          print("There is no such route operated by Citybus.")
          break
        print(f"Stop names for route {route} departing from {list(stop_name_list.values())[0]} to {list(stop_name_list.values())[-1]}:")
        # Create a Pandas DataFrame from the data
        stop_table = {"Stop Names": list(stop_name_list.values()), "Stop IDs": list(stop_name_list.keys()), "Bound": bound}
        stop_table_main[bound] = stop_table
        df = pd.DataFrame(stop_table)
        df_main[bound] = df
        # Create buttons for each row
        buttons = []
        for i in range(len(df_main[bound])):
            button = widgets.Button(description=df_main[bound].iloc[i]['Stop Names'])
            button.layout.width = "300px"  # Adjust the button width here
            buttons.append(button)

        # Create a VBox layout for the buttons
        vbox = widgets.VBox(buttons)

        # Display the buttons
        display(vbox)

        # Add button click event handlers
        for i in range(len(buttons)):
            buttons[i].on_click(lambda event, index=i, bound=bound: handle_click(index, bound))
    else:
        print(f'Error: Unable to retrieve route information for route {route}')

if check:
    stop_name_list_main.append(stop_name_list)

In [ ]:
#bus tracker KMB and LWB only
import requests
import json
import datetime
import pytz
import pandas as pd

check = True
count = 0
route = input("Enter the bus route: ")
directions = ('inbound', 'outbound')
stopid_list_main = []
stop_name_list_main = []
for bound in directions:
  if count == 0 and check == False:
    break
  for count in range(10):
    url = f"https://data.etabus.gov.hk/v1/transport/kmb/route-stop/{route}/{bound}/{count + 1}"
    response = requests.get(url)
    if response.status_code == 200:
          stopid_list = []
          # Parse the JSON response and extract the stop information
          data = json.loads(response.text)
          route_list = data["data"]
          # Display the stop information to the user
          for stop in route_list:
              stopid_list.append(stop['stop'])
          if stopid_list == [] and count == 0:
            print("There is no such route operated by KMB or LWB")
            check = False
            break
          elif stopid_list == []: break
          stopid_list_main.append(stopid_list)
          stop_name_list = {}
          for stop_id in stopid_list:
              url_stop = f"https://data.etabus.gov.hk/v1/transport/kmb/stop/{stop_id}"
              response_stop = requests.get(url_stop)
              if response_stop.status_code == 200:
                  # Parse the JSON response and extract the stop information
                  data_stop = json.loads(response_stop.text)
                  stop_name_list[stop_id] = data_stop["data"]["name_en"]
                  # Display the stop information to the user

          print(f"\n Stop names for route {route} departing from {stop_name_list[stopid_list[0]]} to {stop_name_list[stopid_list[-1]]}:")
              # Create a Pandas DataFrame from the data
          stop_table = {"Stops ID": list(stop_name_list.keys()), "Stop Names": list(stop_name_list.values())}
          df = pd.DataFrame(stop_table)
          # Set the 'Name' column as the index
          df.set_index('Stops ID', inplace=True)
          # Print the DataFrame as a table
          print(df)
          stop_name_list_main.append(stop_name_list)
    else:
          print(f"Error: Unable to retrieve routing for route {route}")

while check == True:
    trace_stop = input("Please input the stop id u would like to trace: ")
    count = 0
    for stopid_lists in stopid_list_main:
      if trace_stop in stopid_lists:
          eta_url = f"https://data.etabus.gov.hk/v1/transport/kmb/stop-eta/{trace_stop}"
          eta_response = requests.get(eta_url)
          if eta_response.status_code == 200:
              retrieved = json.loads(eta_response.text)
              data = retrieved["data"]
              eta_list_o = []
              eta_list_i = []
              for entries in data:
                try:
                  if entries["rmk_en"] != "KMB Cycle" and entries["dir"] == "O" and entries["route"] == route:
                      eta_form = datetime.datetime.fromisoformat(entries["eta"])
                      eta_list_o.append(eta_form)
                  elif entries["rmk_en"] != "KMB Cycle" and entries["dir"] == "I" and entries["route"] == route:
                      eta_form = datetime.datetime.fromisoformat(entries["eta"])
                      eta_list_i.append(eta_form)
                except:
                      print(f'There is no service at the stop by KMB in the coming hour')
              remaining_time_o = []
              remaining_time_i = []
              for eta in eta_list_o:
                  remaining_time_o.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
              for eta in eta_list_i:
                  remaining_time_i.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
              remaining_time_main = (remaining_time_i, remaining_time_o)
              remaining_time_i = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_i]
              remaining_time_o = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_o]
              if remaining_time_i != []:
                for remaining_time in remaining_time_i:
                  if count <= 3:
                    print(f"Remaining time for route {route} to {list((stop_name_list_main[count]).values())[-1]} at stop {stop_name_list_main[count][trace_stop]} is {remaining_time}")
                    count += 1
              elif remaining_time_o != []:
                for remaining_time in remaining_time_o:
                  if count <= 3:
                    print(f"Remaining time for route {route} to {list((stop_name_list_main[count]).values())[-1]} at stop {stop_name_list_main[count][trace_stop]} is {remaining_time}")
                    count += 1
          else:
              print(f'Error: Unable to retrieve ETAs for route {route}')

    check = input("Do you want to trace another Stop? T/F ")
    check = check.upper()
    if check == "T":
        check = True
        continue
    else: break

In [ ]:
#bus tracker KMB and LWB only (v.2)
import requests
import json
import datetime
import pytz
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

check = True
route = input("Enter the bus route: ")
route = route.upper()
directions = ('inbound', 'outbound')
stopid_list_main = []
stop_name_list_main = []


def handle_click(index, service_type):
    # Get the associated row data
    stop_name = df[service_type].iloc[index]['Stop Names']
    stop_id = df[service_type].iloc[index]['Stop IDs']
    list_count = df[service_type].iloc[index]['Count']
    display_eta(stop_name, stop_id, route, service_type, list_count)

def display_eta(stop_name, stop_id, route, service_type, list_count):
    count = 0
    eta_url = f"https://data.etabus.gov.hk/v1/transport/kmb/stop-eta/{stop_id}"
    eta_response = requests.get(eta_url)
    if eta_response.status_code == 200:
        retrieved = json.loads(eta_response.text)
        eta_data = retrieved["data"]
        eta_list = []
        st = service_type.upper()
        for entries in eta_data:
            if entries["route"] == route and str(entries["service_type"]) == service_type[-1] and entries["dir"] == st[0]:
                    eta_form = datetime.datetime.fromisoformat(entries["eta"])
                    eta_list.append(eta_form)
        remaining_time_main = []
        for eta in eta_list:
            remaining_time_main.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
        remaining_time_main = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_main]
        if remaining_time_main != []:
            for remaining_time in remaining_time_main:
                    print(f"Remaining time for route {route} to {list((stop_name_list_main[list_count - 1]).values())[-1]} at stop {stop_name} is {remaining_time}")
        else:
            print(f'There is no service at the stop by KMB in the coming hour')
    else:
        print(f'Error: Unable to retrieve ETAs for route {route}')

stop_table = {}
stop_table_main = {}
df = {}
df_main = {}
count = 0
list_count = 0
for bound in directions:
    if count == 0 and check == False:
        break
    for count in range(10):
        url = f"https://data.etabus.gov.hk/v1/transport/kmb/route-stop/{route}/{bound}/{count + 1}"
        response = requests.get(url)
        if response.status_code == 200:
            stop_name_list = {}
            stopid_list = []
            service_type_list = {}
            # Parse the JSON response and extract the stop information
            data = json.loads(response.text)
            route_list = data["data"]
            # Display the stop information to the user
            for stop in route_list:
                service_type = bound + stop['service_type']
                stop_id = stop['stop']
                url_stop = f"https://data.etabus.gov.hk/v1/transport/kmb/stop/{stop_id}"
                response_stop = requests.get(url_stop)
                if response_stop.status_code == 200:
                    # Parse the JSON response and extract the stop information
                    data_stop = json.loads(response_stop.text)
                    stop_name_list[stop_id] = data_stop["data"]["name_en"]
                    service_type_list[stop_id] = service_type
                    # Display the stop information to the user
            if stop_name_list == {} and count == 0:
                print("There is no such route operated by KMB/LWB.")
                check = False
                break
            elif stop_name_list == {}:
                break
            if stop_name_list != {}:
                stop_name_list_main.append(stop_name_list)
            else: break
            print(f"\nStop names for route {route} \n{list(stop_name_list.values())[0]}\nTO \n{list(stop_name_list.values())[-1]}")
            list_count += 1
            # Create a Pandas DataFrame from the data
            sub_stop_table = {"Stop Names": list(stop_name_list.values()), "Stop IDs": list(stop_name_list.keys()), "Service Type": service_type, "Count" : list_count}
            stop_table[service_type] = sub_stop_table
            sub_df = pd.DataFrame(sub_stop_table)
            df[service_type] = sub_df
            # Create buttons for each row
            buttons = []
            for i in range(len(df[service_type])):
                button = widgets.Button(description=df[service_type].iloc[i]['Stop Names'])
                button.layout.width = "300px"  # Adjust the button width here
                buttons.append(button)

            # Create a VBox layout for the buttons
            vbox = widgets.VBox(buttons)

            # Display the buttons
            display(vbox)

            # Add button click event handlers
            for i in range(len(buttons)):
                buttons[i].on_click(lambda event, index=i, service_type=service_type: handle_click(index, service_type))
        else:
            print(f"Error: Unable to retrieve routing for route {route}")